In [51]:
import matplotlib.pyplot as plt
import vowpalwabbit
import random
import math
import json


In [52]:
def convert_txt_to_json(txt_path, json_out_path):
    json_lines = []

    with open(txt_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith("#"):
                continue

            try:
                cost_part, rest = line.split(":", 1)
                min_max_part, action_feat = rest.split("]:", 1)
                min_val, max_val = map(float, min_max_part.strip("[").split(","))
                action, features_str = action_feat.strip().split(" |", 1)

                # Label for CATs
                label = {
                    "action": float(action),
                    "cost": float(cost_part),
                    # Use uniform PDF over [min, max] for simplicity
                    "pdf_value": 1.0 / (max_val - min_val) if (max_val > min_val) else 1.0
                }

                # Parse features: convert `key:value` to JSON-style
                context = {}
                for feat in features_str.strip().split():
                    if ":" in feat:
                        k, v = feat.split(":")
                        context[k] = float(v)
                    else:
                        context[feat] = 1  # one-hot categorical

                vw_json = {
                    "_label_ca": label,
                    "c": context
                }

                json_lines.append(json.dumps(vw_json))
            except Exception as e:
                print(f"❌ Error parsing line: {line}")
                print(e)

    with open(json_out_path, "w") as out:
        for line in json_lines:
            out.write(line + "\n")

In [54]:

def to_vw_json_example(context, action, cost, pdf_value):
    return json.dumps({
        "_label_ca": {
            "action": action,
            "cost": cost,
            "pdf_value": pdf_value
        },
        "c": context
    })


In [55]:
def run_scheduler_simulation(
    vw,
    num_iterations,
    task_sampler,           # function returning (context_dict, ideal_time)
    cost_function,          # function(context, action) → cost
    min_value,
    max_value,
    do_learn=True,
):
    reward_rate = []
    hits = 0
    cost_sum = 0.0

    for i in range(1, num_iterations + 1):
        # 1. Sample a task context + its ideal scheduled time (for cost calc)
        context, ideal_time = task_sampler()

        # 2. Predict a time of day (float) for the task using VW
        ex = vw.example(json.dumps({"c": context}))
        predicted_time = vw.predict(ex)
        ex.finish()

        # 3. Compute cost from the predicted time
        cost = cost_function(context, predicted_time, ideal_time)
        if cost <= -0.75:
            hits += 1
        cost_sum += cost

        # 4. Learn from the example if training
        if do_learn:
            pdf_value = 1.0 / (max_value - min_value)  # assume uniform exploration for now
            vw_ex = vw.parse(
                to_vw_json_example(context, predicted_time, cost, pdf_value),
                label_type="continuous"
            )
            vw.learn(vw_ex)
            vw.finish_example(vw_ex)

        reward_rate.append(-1 * cost_sum / i)

    return reward_rate, hits

In [56]:
def plot_reward_rate(num_iterations, reward_rate, title):
    plt.show()
    plt.plot(range(1, num_iterations + 1), reward_rate)
    plt.xlabel("num_iterations", fontsize=14)
    plt.ylabel("reward rate", fontsize=14)
    plt.title(title)
    plt.ylim([0, 1])

In [57]:
num_iterations = 5000

num_actions = 32
bandwidth = 1

# Instantiate VW learner
vw = vowpalwabbit.Workspace(
    "--cats "
    + str(num_actions)
    + "  --bandwidth "
    + str(bandwidth)
    + " --min_value 0 --max_value 32 --json --chain_hash --coin --epsilon 0.2 -q :: --quiet"
)
ctr, hits = run_simulation(
    vw, num_iterations, rooms, times_of_day, get_cost, 0, 32, do_learn=True
)
vw.finish()
plot_reward_rate(
    num_iterations, ctr, "reward rate with num_actions = 32 and bandwidth = 1"
)

TypeError: convert_txt_to_json() missing 1 required positional argument: 'json_out_path'